In [27]:
import pandas as pd
import numpy as np
from sklearn import neighbors, datasets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import plot_confusion_matrix, accuracy_score, mean_absolute_error
from sklearn.model_selection import StratifiedShuffleSplit, ShuffleSplit, StratifiedKFold, train_test_split, GridSearchCV
from sklearn.svm                    import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from sklearn.linear_model import ElasticNet
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import fbeta_score, make_scorer

#0. import the csv

dataTest_X= pd.read_csv(r'F:\Documenti\machine_learning_avanzato\esercizio_regressione_soluzioni\data\houses_train.csv') 
#dataTest_X = pd.read_csv('trainingBarHOSP.csv') 


#1.clean the data

#dataTest_X = dataTest_X.drop(['Lat', 'Long','Full Address'], axis=1)
#dataTest_X = dataTest_X.drop(['Unnamed: 0', 'Unnamed: 0.1','DistBar'], axis=1)
#dataTest_X = dataTest_X.drop(['Full Address','DistBar'], axis=1)

# remove zero prices
dataTest_X=dataTest_X[(dataTest_X.price>0)]

# id 3056
dataTest_X=dataTest_X[dataTest_X.id!=3056]

# tolgo id
#dataTest_X = dataTest_X.drop(['id'], axis=1)

# tolgo date
dataTest_X = dataTest_X.drop(['date'], axis=1)

#remove street 
dataTest_X = dataTest_X.drop(['street', 'country','city'], axis=1)


#### check outliers
#compute price_mq
# dataTest_X['price_mq']=dataTest_X.price/dataTest_X.sqft_living

#2. remove bedrooms outliers
var = 'bedrooms'
data = dataTest_X['price']
np.quantile(data, 0.99)

df_3 = dataTest_X[dataTest_X['bedrooms'] == 3]
data_3 = df_3['price']
q99 = np.quantile(data_3, 0.99)
df_3[df_3['price'] > q99]
dataTest_X.loc[(dataTest_X['bedrooms'] == 3)& (dataTest_X['price'] >q99), 'price'] = q99

df_4 = dataTest_X[dataTest_X['bedrooms'] == 4]
data_4 = df_4['price']
q99 = np.quantile(data_4, 0.99)
df_4[df_4['price'] > q99]
dataTest_X.loc[(dataTest_X['bedrooms'] == 4)& (dataTest_X['price'] >q99), 'price'] = q99

df_5 = dataTest_X[dataTest_X['bedrooms'] == 5]
data_5 = df_5['price']
q99 = np.quantile(data_5, 0.99)
df_5[df_5['price'] > q99]
dataTest_X.loc[(dataTest_X['bedrooms'] == 5)& (dataTest_X['price'] >q99), 'price'] = q99

df_7 = dataTest_X[dataTest_X['bedrooms'] == 7]
data_7 = df_7['price']
q99 = np.quantile(data_7, 0.99)
df_7[df_7['price'] > q99]
dataTest_X.loc[(dataTest_X['bedrooms'] == 7)& (dataTest_X['price'] >q99), 'price'] = q99

#3. enrich with HPI #######
HPIfile = pd.read_csv(r'F:\Documenti\machine_learning_avanzato\esercizio_regressione_soluzioni\emanuele\HPI.csv') 
dataTest_X.statezip=dataTest_X.statezip.str.slice(3,9)
dataTest_X.statezip=pd.to_numeric(dataTest_X.statezip)
dataTest_X = pd.merge(dataTest_X, HPIfile, left_on='statezip', right_on='ZIP Code')
dataTest_X = dataTest_X.drop(['ZIP Code'], axis=1) 
# dataTest_X = dataTest_X.drop(['statezip'], axis=1) # va meglio senza

#dataTest_X=dataTest_X.set_index('id')


dataTest_X.head()


,id,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,HPI
0,2315,700000.0,3.0,2.75,2500,7378,1.0,0,0,5,1390,1110,1948,1985,1016.94
1,1386,1595400.0,3.0,3.25,3790,19000,2.0,0,4,3,3790,0,1985,0,1016.94
2,2057,805000.0,5.0,2.50,4600,19831,1.0,0,3,3,2300,2300,1956,2013,1016.94
3,4535,198995.0,3.0,1.00,1410,8053,1.0,0,0,4,1410,0,1951,1999,1016.94
4,3395,425000.0,3.0,2.50,2320,2267,3.0,0,0,3,2320,0,2009,0,1016.94


In [28]:

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(dataTest_X)

# 4.  subdivide the train in train and test

dataTest_y = dataTest_X.price
dataTest_X = dataTest_X.drop(['price'], axis=1)

train_X, test_X, train_y, test_y = train_test_split(dataTest_X , dataTest_y, test_size=0.2, random_state=0, shuffle=True)


# 5. define mape
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_absolute_percentage_error_scorer = make_scorer(mean_absolute_percentage_error, greater_is_better=False)


In [29]:
###### questo ha MLP regressor

from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPRegressor
from sklearn.pipeline import Pipeline

pipe_MLPRegressor = Pipeline([('scaler',  StandardScaler()),('MLPRegressor', MLPRegressor(random_state = 42))])
grid_params_MLPRegressor = [{'MLPRegressor__solver': ['lbfgs'],'MLPRegressor__max_iter': [1000],'MLPRegressor__activation' : ['relu','logistic','tanh'], 'MLPRegressor__hidden_layer_sizes':[(2,), (4,),(2,2),(4,4),(4,2),(10,10),(2,2,2)],}]

grid = GridSearchCV(estimator=pipe_MLPRegressor, param_grid=grid_params_MLPRegressor, cv=10,scoring='r2', verbose=1, n_jobs=-1)
X = train_X.drop(['id'], axis=1)
#X = train_X
grid_result = grid.fit(X, train_y)
print('Best Score with Ridge: ', grid_result.best_score_)
print('Best Params with Ridge: ', grid_result.best_params_)

X = test_X.drop(['id'], axis=1)
#X = test_X
y_preden = grid.predict(X)
print(mean_absolute_percentage_error(test_y,y_preden))
test_X.head()

Fitting 10 folds for each of 21 candidates, totalling 210 fits
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 192 tasks      | elapsed:   31.1s
[Parallel(n_jobs=-1)]: Done 203 out of 210 | elapsed:   32.2s remaining:    1.0s
[Parallel(n_jobs=-1)]: Done 210 out of 210 | elapsed:   32.6s finished
Best Score with Ridge:  0.7233077944141717
Best Params with Ridge:  {'MLPRegressor__activation': 'relu', 'MLPRegressor__hidden_layer_sizes': (4, 4), 'MLPRegressor__max_iter': 1000, 'MLPRegressor__solver': 'lbfgs'}
20.38373402914431


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,HPI
2255,347,3.0,1.75,1890,9920,1.0,0,0,3,1230,660,1944,0,1137.50
566,842,5.0,1.50,3360,5402,2.0,0,0,3,3360,0,2014,0,749.10
981,4321,4.0,2.50,2570,5250,2.0,0,0,3,2570,0,1990,2009,404.58
1009,845,3.0,2.50,2134,1984,2.5,0,0,3,2134,0,2008,0,404.58
2472,2095,3.0,1.50,1860,7963,1.0,0,0,3,1200,660,1963,2008,587.12


In [11]:
def mape(y_true, y_pred):
    assert y_true.ndim==1
    assert y_pred.ndim==1
    return ((y_true - y_pred) / y_true).abs().mean()

In [12]:
print(mape(test_y,y_preden))

0.20194508303227168


In [30]:






##################### prediction on test.csv


Final_test= pd.read_csv(r'F:\Documenti\machine_learning_avanzato\esercizio_regressione_soluzioni\data\houses_test_no_price.csv')

# l'id non viene tolto perche' ci serve 

# tolgo date
Final_test = Final_test.drop(['date'], axis=1)

#remove street 
Final_test = Final_test.drop(['street', 'country','city'], axis=1)

HPIfile = pd.read_csv('F:\Documenti\machine_learning_avanzato\esercizio_regressione_soluzioni\emanuele\HPI.csv') 
Final_test.statezip=Final_test.statezip.str.slice(3,9)
Final_test.statezip=pd.to_numeric(Final_test.statezip)
Final_test = pd.merge(Final_test, HPIfile, left_on='statezip', right_on='ZIP Code')
Final_test = Final_test.drop(['ZIP Code'], axis=1) 
# Final_test = Final_test.drop(['statezip'], axis=1)
Final_test.head()


,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,HPI
0,858,2.0,2.00,1420,9191,1.5,0,2,5,1420,0,1928,1970,1415.69
1,1156,2.0,1.50,840,867,2.0,0,0,3,840,0,2005,0,1415.69
2,1356,2.0,1.00,1040,5527,1.0,0,0,3,1040,0,1951,1994,1415.69
3,1971,3.0,2.00,1463,868,3.0,0,0,3,1463,0,2003,0,1415.69
4,1745,3.0,1.75,1620,4200,1.0,0,0,5,830,790,1945,0,1415.69


In [31]:
X                  = Final_test.drop(['id'], axis=1)
final_y_pred       = grid.predict(X)
Final_test['pred'] = final_y_pred


In [32]:
Final_test.head()

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,HPI,pred
0,858,2.0,2.00,1420,9191,1.5,0,2,5,1420,0,1928,1970,1415.69,713410.296175
1,1156,2.0,1.50,840,867,2.0,0,0,3,840,0,2005,0,1415.69,457937.198967
2,1356,2.0,1.00,1040,5527,1.0,0,0,3,1040,0,1951,1994,1415.69,454641.518131
3,1971,3.0,2.00,1463,868,3.0,0,0,3,1463,0,2003,0,1415.69,515768.016990
4,1745,3.0,1.75,1620,4200,1.0,0,0,5,830,790,1945,0,1415.69,551613.859338


In [33]:
df_test = pd.read_csv(r'F:\Documenti\machine_learning_avanzato\data\houses_test.csv')
df_test = df_test[df_test['price']>0.1]

In [34]:
def mape(y_true, y_pred):
    assert y_true.ndim==1
    assert y_pred.ndim==1
    return ((y_true - y_pred) / y_true).abs().mean()

In [35]:
mape(y_true=df_test['price'], y_pred=Final_test['pred'])

0.6780889985961707

In [26]:
Final_test

,id,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,statezip,HPI,pred
0,858,2.0,2.00,1420,9191,1.5,0,2,5,1420,0,1928,1970,98115,1415.69,875976.960396
1,1156,2.0,1.50,840,867,2.0,0,0,3,840,0,2005,0,98115,1415.69,550310.328349
2,1356,2.0,1.00,1040,5527,1.0,0,0,3,1040,0,1951,1994,98115,1415.69,484984.326133
3,1971,3.0,2.00,1463,868,3.0,0,0,3,1463,0,2003,0,98115,1415.69,557406.892696
4,1745,3.0,1.75,1620,4200,1.0,0,0,5,830,790,1945,0,98115,1415.69,537317.678950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,1279,4.0,1.00,1450,8960,1.0,0,0,3,1450,0,1943,2002,98146,862.29,308906.164088
916,1401,4.0,2.50,2340,5600,2.0,0,0,3,2340,0,1921,2013,98146,862.29,615113.338590
917,3262,2.0,1.00,900,5220,1.0,0,0,4,900,0,1956,0,98146,862.29,239428.135204
918,1944,3.0,2.75,3150,219978,2.0,0,0,4,3000,150,1990,0,98050,596.60,733004.601141


In [12]:
import matplotlib.pyplot as plt
%matplotlib notebook

In [13]:
fig, ax = plt.subplots(1, 1)

<IPython.core.display.Javascript object>

In [14]:
df = df_test[['price']].join(Final_test['pred'])

In [15]:
df['price'] = df['price'].astype('float')

In [16]:
df[['pred']].plot()

<IPython.core.display.Javascript object>

In [17]:
plt.plot(df_test['price'].to_numpy())

<IPython.core.display.Javascript object>

In [19]:
plt.plot([1, 2, 3, 4, 5, 6])

<IPython.core.display.Javascript object>